In [ ]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import traitlets
import matplotlib.pyplot as plt
import random
from matplotlib import animation

%matplotlib inline
#%pylab inline

In [ ]:
#data = pd.ExcelFile("..\..\..\Basketball Tracking Data\toy 1.xlsx")
file = pd.ExcelFile("toy 1.xlsx")
df = file.parse()
#print(df)

In [ ]:
description = df.describe()
print(description)

In [ ]:
game_id = df['GAME_CODE'][0]
num_rows = len(df.index)
num_rows, game_id

In [ ]:
df[0:5]

In [ ]:
testcourt = df[['X_POSITION','Y_POSITION','Z_POSITION']]
testcourt_array=[testcourt['X_POSITION'][::14],testcourt['Y_POSITION'][::14]]
#testcourt
testcourt_array

In [ ]:
#plt.imshow([testcourt['X_POSITION'],testcourt['Y_POSITION']], interpolation='nearest')
fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(testcourt['X_POSITION'][::14], testcourt['Y_POSITION'][::14], c='red')
fig.tight_layout()

In [ ]:
def format_plot(ax):
    ax.set_xlim(-1, 101)
    ax.set_ylim(-1, 51)
    #ax.set_xticks([-2, -1, 0, 1, 2])
    #ax.set_yticks([-2, -1, 0, 1, 2])
    ax.set_xlabel(r"$x$ / a.u.")
    ax.set_ylabel(r"$y$ / a.u.")
    ax.set_aspect('equal')

#def state(t=0, player=0):
def state(t):
    player = 0
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.scatter(testcourt['X_POSITION'][14*t+player], testcourt['Y_POSITION'][14*t+player], c='red')
    format_plot(ax)
    fig.tight_layout()
def state2(t=0):
    player = 0
    plt.imshow([testcourt_array[::14,0],testcourt_array[::14,0]], interpolation='nearest')
    plt.show()

In [ ]:
court_test = widgets.interact(state, t=(0, 10000//14))
state(10)

In [ ]:
maximum = 1000
step = 10
play = widgets.Play(
#     interval=10,
    value=0,
    min=0,
    max=maximum,
    step=step,
    description="Press play",
    disabled=False
)
#slider = widgets.IntSlider(min=0, max=maximum, step=step)
#court_disp = widgets.interactive(state, t=0, player=0)
#widgets.jslink((play, 'value'), (slider, 'value'))
#widgets.HBox([play, slider, court_disp])
court_disp = widgets.interactive(state, t=(0, maximum, step), continuous_update=True)
#court_disp.observe(play, 'value')
#play.observe(court_disp)
widgets.jsdlink((play, 'value'), (court_disp.children[0], 'value'))
#widgets.jslink((play, 'value'), (court_disp.children[0], 'value'))
#widgets.jsdlink((slider, 'value'), (court_disp.children[0], 'value'))
#traitlets.link((play, 'value'), (court_disp.children[0], 'value'))
#widgets.HBox([play, court_disp])
display(play, court_disp)

In [ ]:
display(court_disp)

In [ ]:
caption = widgets.Label(value = 'The values of range1 and range2 are synchronized')
range1, range2 = widgets.IntSlider(description='Range 1'),\
                         widgets.IntSlider(description='Range 2')
l = widgets.jslink((range1, 'value'), (range2, 'value'))
display(caption, range1, range2)

In [ ]:
class AnimatedScatter(object):
    """An animated scatter plot using matplotlib.animations.FuncAnimation."""
    def __init__(self, numpoints=50):
        self.numpoints = numpoints
        self.stream = self.data_stream()

        # Setup the figure and axes...
        self.fig, self.ax = plt.subplots()
        # Then setup FuncAnimation.
        self.ani = animation.FuncAnimation(self.fig, self.update, interval=5, 
                                           init_func=self.setup_plot, blit=True)

    def setup_plot(self):
        """Initial drawing of the scatter plot."""
        x, y, s, c = next(self.stream)
        self.scat = self.ax.scatter(x, y, c=c, s=s, animated=True)
        self.ax.axis([-10, 10, -10, 10])

        # For FuncAnimation's sake, we need to return the artist we'll be using
        # Note that it expects a sequence of artists, thus the trailing comma.
        return self.scat,

    def data_stream(self):
        """Generate a random walk (brownian motion). Data is scaled to produce
        a soft "flickering" effect."""
        data = np.random.random((4, self.numpoints))
        xy = data[:2, :]
        s, c = data[2:, :]
        xy -= 0.5
        xy *= 10
        while True:
            xy += 0.03 * (np.random.random((2, self.numpoints)) - 0.5)
            s += 0.05 * (np.random.random(self.numpoints) - 0.5)
            c += 0.02 * (np.random.random(self.numpoints) - 0.5)
            yield data

    def update(self, i):
        """Update the scatter plot."""
        data = next(self.stream)

        # Set x and y data...
        self.scat.set_offsets(data[:2, :])
        # Set sizes...
        self.scat._sizes = 300 * abs(data[2])**1.5 + 100
        # Set colors..
        self.scat.set_array(data[3])

        # We need to return the updated artist for FuncAnimation to draw..
        # Note that it expects a sequence of artists, thus the trailing comma.
        return self.scat,

    def show(self):
        plt.show()

if __name__ == '__main__':
    a = AnimatedScatter()
    a.show()

In [ ]:
def main():
    numframes = 100
    numpoints = 10
    color_data = np.random.random((numframes, numpoints))
    x, y, c = np.random.random((3, numpoints))

    fig = plt.figure()
    scat = plt.scatter(x, y, c=c, s=100)

    ani = animation.FuncAnimation(fig, update_plot, frames=range(numframes),
                                  fargs=(color_data, scat))
    plt.show()

def update_plot(i, data, scat):
    scat.set_array(data[i])
    return scat,

main()

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML

# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()

ax.set_xlim(( 0, 2))
ax.set_ylim((-2, 2))

line, = ax.plot([], [], lw=2)

# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return (line,)

# animation function. This is called sequentially
def animate(i):
    x = np.linspace(0, 2, 1000)
    y = np.sin(2 * np.pi * (x - 0.01 * i))
    line.set_data(x, y)
    return (line,)

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=20, blit=True)

HTML(anim.to_html5_video())